In [39]:
#import libraries
import os
import subprocess
import pandas as pd
import numpy as np

In [40]:

# Create empty list and dictionary
sequenc=[]
dicti_r={}

# Set initial value for app variable
app=0

# Loop through numbers 1 to 60
for nu in range(1,61):
    # Create empty list for filtered filenames
    filenames_filt=[]
    
    # Get list of all filenames in directory
    filenames=os.listdir("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/rnarobo_results/"+str(nu))
    
    # Filter filenames to only include .txt files
    for fi in filenames:
        if fi[-3:]=="txt":
            filenames_filt.append(fi)

    # Loop through filtered filenames
    for filename in filenames_filt:
        # Open file in read-only mode
        with open(os.path.join("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/rnarobo_results/"+str(nu), filename), 'r') as f:
            # Read all lines in file
            f_l=f.readlines()
            
            # Check if at least one ribo was found
            if len(f_l)>17:
                # Loop through lines in file
                for linea in f_l[11:-6]:
                    # Check if we want to append the sequence (means we found a ribo before)
                    if app==1:
                        sequenc.append(linea)
                        app=0
                    # Check if line is not a separator line
                    if linea[0]!="|":
                        # Split line into list of words
                        line=linea.strip().split()
                        try:
                            # Check if RNA sequence is already in dictionary,  then save the start position of the ribo
                            if line[2] in dicti_r:
                                dicti_r[line[2]].append(int(line[0]))
                            else:
                                dicti_r[line[2]]=[int(line[0])]
                            # Set the app variable to 1: add the next line to sequences
                            app=1 
                        except: 
                            pass

In [ ]:
# Create a new DataFrame with a single column named "rnarobo_seq"
allribos=pd.DataFrame(columns=["rnarobo_seq"])

# Set the values of the "rnarobo_seq" column to the contents of the "sequenc" list
allribos["rnarobo_seq"]=sequenc

# Count the number of unique values in the "rnarobo_seq" column and print the result
len(allribos["rnarobo_seq"].unique())

# Convert the DataFrame to a tab-separated string and store it in the "df1str" variable
df1str=allribos.to_csv(sep="\t",index=False)

# Open a new file named "all_WGS_ribos.csv" in write mode and write the contents of "df1str" to it
f= open("tables_and_results/all_WGS_ribos.csv","w+")
f.write(df1str)
f.close()

In [10]:
# Create an empty dictionary to store the trnascan files
dicti={}

# Loop through numbers 1 to 60
for nu2 in range(1,61):
    # Get list of all filenames in directory
    filenames=os.listdir("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/trnascan/"+str(nu2))
    
    # Create empty list for filtered filenames
    filenames_filt=[]
    
    # Filter filenames to only include .txt files
    for fi in filenames:
        if fi[-3:]=="txt":
            filenames_filt.append(fi)
    
    # Loop through filtered filenames
    for filename in filenames_filt:
        # Open file in read-only mode
        f=open("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/trnascan/"+str(nu2)+"/"+filename, 'r')
        
        # Read all lines in file
        f_l=f.readlines()
        
        # Check if at least one tRNA was found
        if len(f_l)>1:
            # Loop through lines in file
            for line in f_l[3:]:
                # Split line into list of words
                line=line.strip().split()
                
                # Check if tRNA sequence is already in dictionary, then save the start position of the tRNA
                if line[0] in dicti:
                    dicti[line[0]].append(int(line[3]))
                else:
                    dicti[line[0]]=[int(line[3])]

In [ ]:
# Create an empty list to store the matching tRNA and ribozyme sequences
matching=[]

# Loop through keys in the "dicti_r" dictionary
for key in dicti_r:
    # Check if the key is also in the "dicti" dictionary
    if key in dicti:
        # Loop through values in the "dicti_r" dictionary for the current key
        for value in dicti_r[key]:
            # Loop through values in the "dicti" dictionary for the current key
            for value2 in dicti[key]:
                # Check if the difference between the two values is within a range of -5 to 5
                if value-value2 in range(-5,6):
                    # Append the key and both values to the "matching" list
                    matching.append([key,value,value2])

# Create a copy of the "matching" list and store it in the "newm" variable
newm=matching



In [16]:
# Create a new DataFrame with the specified column names
lets_do_this=pd.DataFrame(columns=["ID","trna_nr","trna_start","trna_end","trna_type","codon","ins_start","ins_end","trna_confidence","psuedo","rnarobo_start","rnarobo_end","rnarobo_seq"])

# Set initial values for variables
check=0
oldn=""
duplicates=[]

# Loop through indices of the "newm" list
for nr in range(len(newm)):
    # Get the current match from the "newm" list
    match=newm[nr]
    
    # Set initial value for "lin_df" variable
    lin_df=""
    
    # Check if the current match has a different filename than the previous match
    if match[0]!=oldn:
        # Open the corresponding tRNA and rRNA files
        trnaf=open("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/together/"+match[0].split(".")[0]+"trna_result.txt")
        trna=trnaf.readlines()
        trnaf.close()
        
        rnarobof=open("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/together/"+match[0].split(".")[0]+"_rnarobo_result.txt")
        rnarobo=rnarobof.readlines()
        rnarobof.close()
        
    # Loop through lines in the tRNA file
    for line in trna[3:]:
        # Split line into list of words
        line=line.strip().split()
        
        # Check if the current line matches the current match
        if match[0]==line[0]:
            if len(lin_df.split())==0:
                if str(match[2])==line[3]:
                    # Append the line to "lin_df"
                    lin_df+=" ".join(line)+" "
                    if lin_df[-2]!="o":
                        lin_df+="NA "
            else: 
                duplicates.append(line)    
    
    # Loop through lines in the rRNA file
    for line in rnarobo[11:-6]:
        # Split line into list of words
        line=line.strip().split()
        
        # Check if the current line matches the current match
        if len(line)>1:
            if match[0]==line[2]:
                if str(match[1])==line[0]:
                    # Append the first two words of the line to "lin_df"
                    lin_df+=" ".join(line[:2])
                    lin_df+=" "
                    check=1
    
    # Set the value of "oldn" to the current match's filename
    oldn=match[0]
    
    # Split "lin_df" into a list of words
    lin_f=lin_df.split()
    
    # Add the list of words as a new row to the "lets_do_this" DataFrame
    lets_do_this.loc[nr]=np.array(lin_f)

In [ ]:
# Create a copy of the "lets_do_this" DataFrame
fff=lets_do_this

# Count the number of occurrences of each tRNA type and store the result in the "y" variable
y=fff["trna_type"].value_counts()

# Add two new columns to the DataFrame for the tRNA sequence and ribozyme control sequence
fff["trnaseq"]= " "
fff["ribozyme_control"]= " "

# Loop through rows in the DataFrame
for ID in range(len(fff)):
    # Get the current match from the DataFrame
    match=fff.iloc[ID,0]
    
    # Open the corresponding rRNA file
    rnarobof=open("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/together/"+match.split(".")[0]+"_seqs_rnarobo_result.txt")
    rnarobo=rnarobof.readlines()
    rnarobof.close()
    
    # Extract the rRNA sequence corresponding to the current match
    sequ=""
    check=0
    for num in rnarobo:
        if num[0]==">":
            check=0
        if check==1:
            sequ+=num.strip()
        if match==num.split(" ")[0][1:]:
            check=1
    
    # Extract the tRNA and rRNA sequences from the full rRNA sequence
    start=int(fff.iloc[ID,2])-1
    end=int(fff.iloc[ID,3])
    start2=int(fff.iloc[ID,10])-1
    end2=int(fff.iloc[ID,11])
    if start < end:
        trna=sequ[start:end]
        rnarobo=sequ[start2:end2]
        fff.iloc[ID,-2]=trna
        fff.iloc[ID,-1]=rnarobo
    if end < start:
        # Reverse the sequence and transcribe it back into the correct format
        end-=1
        start+=1
        end2-=1
        start2+=1
        trna_rev=sequ[end:start]
        ribo_rev=sequ[end2:start2]
        trna=[]
        rnarobo=[]
        for nuc in range(len(trna_rev)-1,-1,-1):
            if trna_rev[nuc]=="A":
                trna.append("T")
            if trna_rev[nuc]=="T":
                trna.append("A")
            if trna_rev[nuc]=="N":
                trna.append("N")
            if trna_rev[nuc]=="G":
                trna.append("C")
            if trna_rev[nuc]=="C":
                trna.append("G")
        for nuc in range(len(ribo_rev)-1,-1,-1):
            if ribo_rev[nuc]=="A":
                rnarobo.append("T")
            if ribo_rev[nuc]=="T":
                rnarobo.append("A")
            if ribo_rev[nuc]=="G":
                rnarobo.append("C")
            if ribo_rev[nuc]=="C":
                rnarobo.append("G")
            if ribo_rev[nuc]=="N":
                rnarobo.append("N")
        trna="".join(trna)
        rnarobo="".join(rnarobo)
        fff.iloc[ID,-2]=trna
        fff.iloc[ID,-1]=rnarobo
    
# Check if the ribozyme control sequence matches the extracted rRNA sequence
for x in range(len(fff)):
    seq1=fff.loc[x,"rnarobo_seq"]
    seq3="".join(seq1.split("|"))
    seq2=fff.loc[x,"ribozyme_control"]
    if seq2!=seq3:
        print("something wrong",x)
        print(seq2)
        print(seq3)

In [ ]:
# Convert the DataFrame "ft" to a tab-separated string and store it in the "df1str" variable
df1str=ft.to_csv(sep="\t",index=False)

# Open the file "WGS_ribo_trna.csv" in write mode and write the contents of "df1str" to it
f= open("tables_and_results/WGS_ribo_trna.csv","w+")
f.write(df1str)
f.close()

# Count the number of unique ribozymes in the "ft" DataFrame and print the result
len(ft["ribozyme_control"].unique())

# Count the number of unique tRNAs in the "ft" DataFrame and print the result
len(ft["trnaseq"].unique())